In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по свиньям.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Свиньи
243,АЛМАТИНСКАЯ ОБЛАСТЬ,2015-04-01,1882.65
477,АТЫРАУСКАЯ ОБЛАСТЬ,2024-10-01,0.00
602,ГАЛМАТЫ,2015-03-01,4.50
1809,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2021-06-01,1317.76
1742,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2015-11-01,776.33
557,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2021-06-01,900.52
2017,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2022-03-01,34.80
619,ГАЛМАТЫ,2016-08-01,0.54
248,АЛМАТИНСКАЯ ОБЛАСТЬ,2015-09-01,2137.15
1440,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2018-06-01,11.52


In [3]:
regions = df['Регион'].unique()
target   = "Свиньи"
horizon  = 3
epsilon = 1e-6

In [4]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [5]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[446.27, 549.78, 698.87]","[314.19, 565.5, 523.64]",127.02,107.68,26.09
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[456.62, 581.37, 475.28]","[565.5, 523.64, 515.15]",74.78,68.83,12.67
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[650.31, 531.38, 416.75]","[523.64, 515.15, 309.94]",96.12,83.24,20.60
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[488.99, 383.76, 1161.83]","[515.15, 309.94, 797.36]",215.23,154.82,24.87
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[391.91, 1186.3, 1079.08]","[309.94, 797.36, 945.29]",242.14,201.57,29.79
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[2.17, 3.09, 1.44]","[0.0, 5.1, 2.2]",1.77,1.65,inf
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[1.06, 0.25, 0.09]","[5.1, 2.2, 0.5]",2.60,2.13,83.54
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[1.26, 0.96, 1.66]","[2.2, 0.5, 1.6]",0.60,0.49,45.93
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[1.34, 2.18, 4.94]","[0.5, 1.6, 5.9]",0.81,0.79,73.43


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,178.62,145.06,27.04
1,АКТЮБИНСКАЯ ОБЛАСТЬ,20.54,17.20,22.31
2,АЛМАТИНСКАЯ ОБЛАСТЬ,99.52,87.05,92.74
3,АТЫРАУСКАЯ ОБЛАСТЬ,1.25,0.89,NaN
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,77.43,70.38,17.01
5,ГАЛМАТЫ,0.53,0.40,NaN
6,ГАСТАНА,0.08,0.07,NaN
7,ГШЫМКЕНТ,2.33,2.06,56.16
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,13.55,11.04,37.39
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,27.47,22.69,13.50


In [ ]:
# res_hw[res_hw['Регион'] == 'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ']

,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
120,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[7.26, 12.12, 14.77]","[3.9, 68.6, 3.6]",33.29,23.67,159.62
121,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[10.91, 13.44, 6.03]","[68.6, 3.6, 5.0]",33.79,22.85,125.98
122,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[18.26, 8.46, 25.66]","[3.6, 5.0, 4.4]",15.04,13.12,319.82
123,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[6.03, 19.49, 11.56]","[5.0, 4.4, 2.6]",10.15,8.36,236.05
124,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[19.16, 11.36, 14.47]","[4.4, 2.6, 3.1]",11.89,11.63,346.45
125,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[10.29, 12.91, 9.57]","[2.6, 3.1, 4.4]",7.79,7.56,243.21
126,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[11.01, 8.06, 3.38]","[3.1, 4.4, 4.6]",5.08,4.27,121.70
127,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[3.77, 1.34, 5.56]","[4.4, 4.6, 13.6]",5.02,3.97,48.07
128,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[1.46, 5.88, 2.8]","[4.6, 13.6, 8.3]",5.76,5.45,63.74
129,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2024-10,2024-12,"[8.01, 3.98, 4.15]","[13.6, 8.3, 5.2]",4.12,3.65,37.79


## SARIMA

In [7]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(0, 0, 1)","(0, 1, 0, 12)",268.55,264.64,61.98,"[606.14, 765.78, 825.33]","[314.19, 565.5, 523.64]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(1, 0, 0)","(0, 1, 1, 12)",118.53,104.44,19.72,"[502.57, 707.31, 581.87]","[565.5, 523.64, 515.15]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(0, 0, 1)","(0, 1, 1, 12)",194.04,177.87,41.44,"[801.02, 603.33, 477.97]","[523.64, 515.15, 309.94]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(1, 0, 1)","(0, 1, 1, 12)",252.70,181.48,28.90,"[480.7, 391.25, 1226.04]","[515.15, 309.94, 797.36]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(2, 0, 0)","(0, 1, 1, 12)",343.19,290.93,41.58,"[410.85, 1333.73, 1180.78]","[309.94, 797.36, 945.29]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 1, 1)","(0, 1, 1, 12)",2.05,1.62,131130486357634351890432.00,"[1.67, 1.97, 2.13]","[0.0, 5.1, 2.2]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(0, 1, 1)","(0, 1, 0, 12)",3.00,2.34,89.42,"[0.19, 0.57, 0.01]","[5.1, 2.2, 0.5]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(1, 1, 2)","(0, 1, 0, 12)",1.01,0.89,73.90,"[1.53, 0.02, 0.06]","[2.2, 0.5, 1.6]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(1, 1, 2)","(0, 1, 0, 12)",3.41,2.56,95.73,"[0.03, 0.06, 0.22]","[0.5, 1.6, 5.9]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,212.60,176.70,33.15
1,АКТЮБИНСКАЯ ОБЛАСТЬ,20.87,18.46,23.86
2,АЛМАТИНСКАЯ ОБЛАСТЬ,95.45,85.03,115.71
3,АТЫРАУСКАЯ ОБЛАСТЬ,1.40,0.83,172076958362723516416.00
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,132.36,120.34,29.98
5,ГАЛМАТЫ,0.16,0.09,65416605469811507200.00
6,ГАСТАНА,0.03,0.02,291197655752696397824.00
7,ГШЫМКЕНТ,0.43,0.37,9.06
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,8.70,6.70,19.91
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,26.49,23.35,13.43


## Facebook Prophet

In [8]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


16:20:18 - cmdstanpy - INFO - Chain [1] start processing
16:20:18 - cmdstanpy - INFO - Chain [1] done processing
16:20:18 - cmdstanpy - INFO - Chain [1] start processing
16:20:18 - cmdstanpy - INFO - Chain [1] done processing
16:20:18 - cmdstanpy - INFO - Chain [1] start processing
16:20:19 - cmdstanpy - INFO - Chain [1] done processing
16:20:19 - cmdstanpy - INFO - Chain [1] start processing
16:20:19 - cmdstanpy - INFO - Chain [1] done processing
16:20:19 - cmdstanpy - INFO - Chain [1] start processing
16:20:19 - cmdstanpy - INFO - Chain [1] done processing
16:20:19 - cmdstanpy - INFO - Chain [1] start processing
16:20:20 - cmdstanpy - INFO - Chain [1] done processing
16:20:20 - cmdstanpy - INFO - Chain [1] start processing
16:20:20 - cmdstanpy - INFO - Chain [1] done processing
16:20:20 - cmdstanpy - INFO - Chain [1] start processing
16:20:20 - cmdstanpy - INFO - Chain [1] done processing
16:20:21 - cmdstanpy - INFO - Chain [1] start processing
16:20:21 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,154.54,131.51,32.03,"[479.22, 585.19, 733.46]","[314.19, 565.5, 523.64]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,100.90,74.97,14.37,"[563.34, 688.53, 572.99]","[565.5, 523.64, 515.15]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,122.43,113.86,27.46,"[683.47, 567.91, 438.94]","[523.64, 515.15, 309.94]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,260.17,196.69,33.26,"[555.04, 426.72, 1230.77]","[515.15, 309.94, 797.36]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,273.16,241.40,36.26,"[420.75, 1212.34, 1143.69]","[309.94, 797.36, 945.29]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,2.48,2.25,inf,"[1.26, 1.44, 0.36]","[0.0, 5.1, 2.2]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,1.87,1.60,66.17,"[2.39, 0.45, 0.83]","[5.1, 2.2, 0.5]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,1.20,1.06,71.25,"[0.45, 0.83, 0.52]","[2.2, 0.5, 1.6]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,3.06,2.22,83.38,"[1.01, 0.64, 0.71]","[0.5, 1.6, 5.9]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,186.54,151.53,27.95
1,АКТЮБИНСКАЯ ОБЛАСТЬ,52.40,43.36,61.17
2,АЛМАТИНСКАЯ ОБЛАСТЬ,149.96,132.93,191.75
3,АТЫРАУСКАЯ ОБЛАСТЬ,1.40,0.83,NaN
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,123.56,117.51,28.19
5,ГАЛМАТЫ,0.16,0.09,NaN
6,ГАСТАНА,0.02,0.01,NaN
7,ГШЫМКЕНТ,2.41,2.03,42.68
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,14.44,11.27,32.11
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,28.55,25.26,14.54


In [9]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW                MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ    27.04                      33.15         27.95          HW
           АКТЮБИНСКАЯ ОБЛАСТЬ    22.31                      23.86         61.17          HW
           АЛМАТИНСКАЯ ОБЛАСТЬ    92.74                     115.71        191.75          HW
            АТЫРАУСКАЯ ОБЛАСТЬ      NaN   172076958362723516416.00           NaN      SARIMA
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    17.01                      29.98         28.19          HW
                       ГАЛМАТЫ      NaN    65416605469811507200.00           NaN      SARIMA
                       ГАСТАНА      NaN   291197655752696397824.00           NaN      SARIMA
                      ГШЫМКЕНТ    56.16                       9.06         42.68      SARIMA
            ЖАМБЫЛСКАЯ ОБЛАСТЬ    37.39                      19.91         32.11      SARIMA
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    13.50                      13.43    